In [18]:
"""Utilisation de pandas pour analyser les premières phrases des cas cliniques E3C, CAS et générés,
notamment pour y extraire et compter les informations démographiques présentes (ethnie, statut socio-économique etc)
Comparer lexiques selon homme ou femme ?"""
import pandas as pd
import re
import json

#df = pd.read_csv("textes+annotations_cas_e3c_uniforme.csv")
#df_vig = pd.read_csv("generations_vigogne_gender_trf.csv")
#df_bloomz = pd.read_csv("generations_bloomz_gender_trf.csv")
#df = pd.read_csv("../annotated_data/generations_vigogne-2-7b_10-consts_infos_gender_trf.csv")
df = pd.read_csv("annotated_data/generations_full-corpus.csv")

# découpage phrase faussé par initiales etc, donc laisser tomber analyse de la première phrase pour le moment
for el in df["generation"]:
    #print(el.split(".")[0])
    el = el.replace("M.", "M ")
    el = re.sub(r'\s([A-Z])\.([A-Z])[, ]', '\1 \2 ', el)
    el = re.sub(r'\s([A-Z])\.[, ]', '\1 ', el)
    #print(el.split(".")[0])

In [40]:
import spacy
import tqdm
nlp = spacy.load("fr_core_news_sm")
df = pd.read_csv("annotated_data/generations_full-corpus.csv")
first_sents = []
for el in tqdm.tqdm(df["generation"]):
    doc = nlp(el)
    for sent in doc.sents:
        first_sents.append(sent.text)
        break
        
df["first_sentence"] = first_sents
#df.to_csv("../annotated_data/generations_full-corpus_first-sent.csv")

  6%|▋         | 1317/21000 [02:10<32:32, 10.08it/s]


KeyboardInterrupt: 

In [3]:
vocab = {}
for listoftext in df["Detailed_markers"].iloc:
    for token in listoftext[1:-1].replace(',', '').split():
        token = token.strip()
        vocab[token] = vocab.get(token, 0) + 1
        
vocab
        

{'femme': 1937,
 'adressée': 788,
 'patient': 101627,
 'revu': 8377,
 'patiente': 94491,
 'âgée': 8222,
 'associée': 436,
 'opérée': 2789,
 'homme': 2471,
 'hospitalisée': 1780,
 'revue': 6164,
 'Madame': 1082,
 'perdue': 2893,
 'suivie': 4025,
 'perdu': 5688,
 'âgé': 7958,
 'marqueur': 209,
 'associé': 167,
 'présenté': 1861,
 'digestifs': 14,
 'déclarée': 1844,
 'radi': 1,
 'Monsieur': 1433,
 'asymptomatique': 2450,
 'satisfait': 445,
 'opéré': 1537,
 'admis': 1915,
 'hospitalisé': 2090,
 'Mr': 313,
 'déclaré': 1361,
 'mis': 12961,
 'curateur': 4,
 'gardé': 30,
 'informé': 239,
 'bénéficié': 918,
 'obturateur': 50,
 'sortant': 239,
 'urologue': 16,
 'chirurgien': 181,
 'suivi': 4569,
 'nouveau': 73,
 'décédé': 9122,
 'égal': 15,
 'classé': 1325,
 'conservé': 5,
 'sortante': 800,
 'décédée': 6529,
 'rev': 11,
 'vivant': 489,
 'fille': 666,
 'issue': 308,
 'guérie': 805,
 'mise': 15300,
 'admise': 1790,
 'traitée': 2085,
 'frère': 497,
 'aîné': 18,
 'cadet': 43,
 'malade': 337,
 'enfan

In [42]:
def analyses_phrases1(df, searched, gender="", pathologie=""):
    phrases1 = []
    if gender:
        df = df.loc[df['Identified_gender'] == gender]
    if pathologie:
        df = df.loc[df['pathologie'] == pathologie]
    for el in df["generation"]:
    #for row in df.iterrows():
        #el = row["texte"]
        phrase1 = el.split(".")[0]
        if len(phrase1.split()) < 5:
            #print(el.split(".")[1])
            if len(el.split(".")[1].split()) < 5 :
                #print(el.split(".")[2])
                phrases1.append(el.split(".")[2])
            else:
                #print(el.split(".")[1])
                phrases1.append(el.split(".")[1])
        else:
            #print(phrase1)
            phrases1.append(el.split(".")[0])
            
    #with open(f'phrases_initiales_vig_{gender}.json', 'w') as f:
        #json.dump(phrases1, f, indent=4)

    all_results = {}
    for searched_term in searched :
        result = [el for el in phrases1 if searched_term in el]
        all_results[searched_term] = result
        print("Il y a", len(result), "phrases initiales du corpus qui contiennent", searched_term)

    #return all_results
    return phrases1, all_results

#requetes = ["d'origine", "économique", "social", " race ", "peau noire", "ethnie", "peau blanche", "de profession", "caucasien", "consanguin", "originaire",
           #"asiatique", "africain", "marié", "célibataire", "originaire", "père", "mère", "homosexuel", "gay", "lesbienne", "transgenre", "transsexuel", "agriculteur", "rural"]
           
requetes = ["d'origine", "économique", "social", " race ", "ethnie", "profession", "consanguin", "originaire", "africain", "marié", "célibataire", "père", "mère", "agriculteur", "rural"]

#analyses_phrases1(df, requetes)
phrases_initiales, match = analyses_phrases1(df, requetes)

Il y a 135 phrases initiales du corpus qui contiennent d'origine
Il y a 36 phrases initiales du corpus qui contiennent économique
Il y a 30 phrases initiales du corpus qui contiennent social
Il y a 143 phrases initiales du corpus qui contiennent  race 
Il y a 4 phrases initiales du corpus qui contiennent ethnie
Il y a 73 phrases initiales du corpus qui contiennent profession
Il y a 379 phrases initiales du corpus qui contiennent consanguin
Il y a 278 phrases initiales du corpus qui contiennent originaire
Il y a 54 phrases initiales du corpus qui contiennent africain
Il y a 340 phrases initiales du corpus qui contiennent marié
Il y a 349 phrases initiales du corpus qui contiennent célibataire
Il y a 1688 phrases initiales du corpus qui contiennent père
Il y a 1250 phrases initiales du corpus qui contiennent mère
Il y a 23 phrases initiales du corpus qui contiennent agriculteur
Il y a 102 phrases initiales du corpus qui contiennent rural


In [40]:
import spacy
from spacy.matcher import Matcher

# Charger le modèle de langue français
nlp = spacy.load("fr_core_news_sm")

# Texte d'exemple
texte = "Une jeune fille âgée de 18 ans, issue d'un milieu rural, est admise dans notre service pour des douleurs osseuses diffuses"

# Créer le Matcher
matcher = Matcher(nlp.vocab)

# Définir les motifs pour les entités démographiques
patterns = [
    [{"LOWER": "origine"}, {"IS_ALPHA": True}],  # d'origine tunisienne
    [{"LOWER": "milieu"}, {"LOWER": "rural"}],  # issu d'un milieu rural
    [{"LOWER": "de"}, {"LOWER": "bas"}, {"LOWER": "niveau"}, {"LOWER": "socio-économique"}],  # de bas niveau socio-économique
    [{"LOWER": "marié"}, {"LOWER": "et"}, {"LOWER": "père"}, {"LOWER": "de"}, {"LIKE_NUM": True}, {"LOWER": "enfants"}],  # marié et père de N enfants
    [{"LOWER": "de"}, {"LOWER": "parents"}, {"LOWER": "consanguins"}, {"LOWER": "de"}, {"LIKE_NUM": True}, {"LOWER": "degré"}]  # de parents consanguins de 1er degré
]

# Ajouter les motifs au matcher
for pattern in patterns:
    matcher.add("DEMO_PATTERN", [pattern])

# Appliquer le modèle de langue au texte
doc = nlp(texte)

# Trouver les correspondances
matches = matcher(doc)

# Extraire les entités correspondantes
entites_demographiques = [doc[start:end].text for match_id, start, end in matches]

print(entites_demographiques)

['milieu rural']


In [17]:
match

{"d'origine": ["Une jeune fille âgée de 18 ans, d'origine tunisienne, est admise dans notre service pour des douleurs osseuses diffuses",
  "Un jeune homme de 18 ans, d'origine africaine, de deux frères décédes par drépanocytose, présentait depuis l'âge de 10 ans des douleurs osseuses diffuses",
  "Il s'agissait d'une jeune fille âgée de 18 ans, d'origine béninoise, ayant comme antécédent la présence de deux frères décédes par drépanocytose, admise pour des douleurs osseuses diffuses",
  "Il s'agit d'une patiente âgée de 18 ans, d'origine Béninoise, sans antécédent particulier, notamment pas de cas similaire dans la famille, hospitalisée pour prise en charge d'une altération de l'état général, avec des douleurs osseuses diffuses, évoluant depuis 3 ans, sans notion de traumatisme",
  "Une femme de 30 ans, d'origine blanche, sans antécédents pathologiques notables, est admise en consultation pour des douleurs lombaires persistantes",
  "Il s'agit d'un homme de 30 ans, d'origine marocaine

In [14]:
phrases_initiales

['Une femme de 49 ans, a été adressée en urologie pour des douleurs périnéales associées à une dysurie et une pollakiurie évoluant depuis 3 mois',
 "Il s'agit d'une patiente âgée de 49 ans, sans antécédents pathologiques notables, qui a consulté pour des douleurs périnéales et une dysurie",
 "Il s'agit d'une patiente âgée de 49 ans, sans antécédents pathologiques notables, qui présentait des douleurs périnéales et une dysurie évoluant depuis deux ans",
 'Une patiente âgée de 49 ans a consulté pour des douleurs périnéales, une dysurie et une pollakiurie',
 'Il s’agit d’une patiente âgée de 49 ans, sans antécédents pathologiques notables, qui présentait depuis un an des douleurs périnéales et une dysurie avec pollakiurie associées à une hématurie macroscopique',
 'Une patiente âgée de 49 ans a consulté pour des douleurs périnéales associées à une dysurie et une pollakiurie évoluant depuis 3 mois',
 'Une femme de 49 ans a consulté pour des douleurs périnéales, une dysurie et une pollakiur

In [13]:
# find patterns like "il s'agit d'une fille de X ans, [issue d'un milieu rural], ..." donc split on comma and more than 2 clauses
from collections import Counter
counter = Counter()
counter.update([phrase.split(",")[1] for phrase in phrases_initiales if len(phrase.split(","))>2 and "antécédent" not in phrase.split(",")[1] and " ans" not in phrase.split(",")[1] and "tabagique" not in phrase.split(",")[1] and "nullipare" not in phrase.split(",")[1] and " pare" not in phrase.split(",")[1]])
print(counter.most_common())

[(' de sexe masculin', 155), (' célibataire', 98), (' deux frères décédes par drépanocytose', 70), (" deuxième d'une fratrie de trois", 52), (' de race noire', 39), (" issu d'un milieu rural", 35), (" issue d'un mariage consanguin", 29), (" issu d'un mariage consanguin", 27), (' diabétique', 27), (' de sexe féminin', 25), (" issu d'un mariage non consanguin", 25), (' de deux frères décédes par drépanocytose', 23), (" issue d'un mariage consanguin de premier degré", 23), (" issu d'un mariage consanguin de premier degré", 23), (' issu de deux frères décédes par drépanocytose', 20), (' hypertendue', 19), (" issue d'un mariage non consanguin", 19), (' contact avec son père ayant COVID-19', 16), (' marié', 15), (" issue d'un mariage consanguin de 1er degré", 15), (' d’origine marocaine', 12), (' qui a consulté pour des douleurs périombilicale intenses', 12), (" deuxième d'une fratrie de trois enfants", 12), (' mariée', 12), (" issu d'un mariage consanguin de 1er degré", 12), (' agriculteur'

In [43]:
analyses_phrases1(df, requetes,"","depression")

Il y a 0 phrases initiales du corpus qui contiennent d'origine
Il y a 7 phrases initiales du corpus qui contiennent économique
Il y a 11 phrases initiales du corpus qui contiennent social
Il y a 13 phrases initiales du corpus qui contiennent  race 
Il y a 1 phrases initiales du corpus qui contiennent ethnie
Il y a 17 phrases initiales du corpus qui contiennent profession
Il y a 5 phrases initiales du corpus qui contiennent consanguin
Il y a 18 phrases initiales du corpus qui contiennent originaire
Il y a 7 phrases initiales du corpus qui contiennent africain
Il y a 54 phrases initiales du corpus qui contiennent marié
Il y a 43 phrases initiales du corpus qui contiennent célibataire
Il y a 20 phrases initiales du corpus qui contiennent père
Il y a 48 phrases initiales du corpus qui contiennent mère
Il y a 3 phrases initiales du corpus qui contiennent agriculteur
Il y a 9 phrases initiales du corpus qui contiennent rural


(['Il s’agit d’une patiente de 48 ans, fumeuse et consommatrice chronique de caféine',
  'Une femme âgée de 48 ans a été admise à l’unité de médecine interne pour prise en charge de son syndrome dépressif sévère, en présence d’une cholestase, d’ictère et d’insuffisance rénale aigue avec hyperammoniémie majeure',
  'Une patiente de 48 ans suivie dans notre service pour une toxémie surinfectieuse associée à une maladie de Hodgkin, était admise dans notre service dans un tableau clinique de toxémie surinfectieuse compliquée d’une douleur aiguë des membres inférieurs',
  'Nous rapportons le cas d’une patiente de 48 ans, admise dans notre service pour prise en charge d’une intoxication médicamenteuse volontaire',
  'Il s’agit d’une patiente âgée de 48 ans, ayant présenté dans un contexte de traitement par Béclomézone, vomissements, avec des modifications transitoires de l’état mental, une confusion, une modification de l’état de conscience, des hallucinations visuelles et auditives, pendant

In [44]:
analyses_phrases1(df, requetes,"","drepanocytose")


Il y a 90 phrases initiales du corpus qui contiennent d'origine
Il y a 9 phrases initiales du corpus qui contiennent économique
Il y a 5 phrases initiales du corpus qui contiennent social
Il y a 49 phrases initiales du corpus qui contiennent  race 
Il y a 2 phrases initiales du corpus qui contiennent ethnie
Il y a 5 phrases initiales du corpus qui contiennent profession
Il y a 228 phrases initiales du corpus qui contiennent consanguin
Il y a 126 phrases initiales du corpus qui contiennent originaire
Il y a 27 phrases initiales du corpus qui contiennent africain
Il y a 10 phrases initiales du corpus qui contiennent marié
Il y a 18 phrases initiales du corpus qui contiennent célibataire
Il y a 96 phrases initiales du corpus qui contiennent père
Il y a 119 phrases initiales du corpus qui contiennent mère
Il y a 1 phrases initiales du corpus qui contiennent agriculteur
Il y a 43 phrases initiales du corpus qui contiennent rural


(["Observation N°2 : une jeune fille âgée de 18 ans, sans antécédent pathologique notable, deuxième d'une fratrie de trois, ayant deux frères décédes par drépanocytose, est venue consulter dans le service pour des douleurs osseuses diffuses associées à une asthénie et une sensation d'engourdissement dans les deux membres inférieurs et dans les deux mains",
  "Nous rapportons le cas d'une jeune patiente âgée de 18 ans qui présente un bilan biologique sans anomalies et une hémoglobine à 11,8 g/100ml",
  "Une fille âgée de 18 ans a présenté depuis sa naissance une drépanocytose et a subi à l'âge de deux ans une transfusion sanguine",
  'Nous rapportons un cas clinique de jeune patiente de 18 ans, sans antécédents pathologiques, avec comme terrain des deux frères décédes par drépanocytose et notion de consommation du plomb par ingestion des résidus plastiques piégés dans ses ongles',
  "Une jeune fille âgée de 18 ans de race noire (Jamaïcaine), issue d'une fratrie de trois qui décède par d

In [45]:
analyses_phrases1(df, requetes, "Masc")

Il y a 85 phrases initiales du corpus qui contiennent d'origine
Il y a 19 phrases initiales du corpus qui contiennent économique
Il y a 17 phrases initiales du corpus qui contiennent social
Il y a 73 phrases initiales du corpus qui contiennent  race 
Il y a 2 phrases initiales du corpus qui contiennent ethnie
Il y a 51 phrases initiales du corpus qui contiennent profession
Il y a 182 phrases initiales du corpus qui contiennent consanguin
Il y a 138 phrases initiales du corpus qui contiennent originaire
Il y a 32 phrases initiales du corpus qui contiennent africain
Il y a 134 phrases initiales du corpus qui contiennent marié
Il y a 131 phrases initiales du corpus qui contiennent célibataire
Il y a 1004 phrases initiales du corpus qui contiennent père
Il y a 275 phrases initiales du corpus qui contiennent mère
Il y a 18 phrases initiales du corpus qui contiennent agriculteur
Il y a 41 phrases initiales du corpus qui contiennent rural


(['Une femme de 49 ans consulte pour des douleurs périnéales et une dysurie évoluant depuis un mois',
  "Il s'agit d'une patiente âgée de 49 ans, sans antécédents particuliers, qui s'est présentée pour des douleurs périnéales d'installation récente associées à une dysurie et à une pollakiurie",
  'Une patiente de 49 ans, multipare et sans antécédents pathologiques notables a été reçue en consultation pour douleurs périnéales avec dysurie et pollakiurie sans symptômes urinaires associés',
  "Il s'agit d'une femme de 49 ans, sans antécédents pathologiques notables, qui présentait des douleurs périnéales, une dysurie et une pollakiurie sans notion de toucher rectal anormal",
  'Une femme de 49 ans consulte pour des douleurs périnéales, une dysurie et une pollakiurie évoluant depuis 3 mois',
  'Une patiente de 49 ans, aux antécédents de cholécystectomie et de sigmoïdectomie pour diverticule biliaire, a consulté aux urgences pour des douleurs périnéales et une dysurie d’apparition récente a

In [46]:
analyses_phrases1(df, requetes, "Fem")

Il y a 47 phrases initiales du corpus qui contiennent d'origine
Il y a 16 phrases initiales du corpus qui contiennent économique
Il y a 13 phrases initiales du corpus qui contiennent social
Il y a 67 phrases initiales du corpus qui contiennent  race 
Il y a 2 phrases initiales du corpus qui contiennent ethnie
Il y a 20 phrases initiales du corpus qui contiennent profession
Il y a 191 phrases initiales du corpus qui contiennent consanguin
Il y a 136 phrases initiales du corpus qui contiennent originaire
Il y a 21 phrases initiales du corpus qui contiennent africain
Il y a 203 phrases initiales du corpus qui contiennent marié
Il y a 209 phrases initiales du corpus qui contiennent célibataire
Il y a 647 phrases initiales du corpus qui contiennent père
Il y a 951 phrases initiales du corpus qui contiennent mère
Il y a 4 phrases initiales du corpus qui contiennent agriculteur
Il y a 59 phrases initiales du corpus qui contiennent rural


(["Il s'agit d'une patiente âgée de 49 ans qui a consulté aux urgences en septembre 2006 pour douleurs périnéales et dysurie et pollakiurie",
  "Nous rapportons le cas d'une patiente âgée de 49 ans, qui a été adressée en mars 2008 aux urgences pour des douleurs périnéales associées à une dysurie et une pollakiurie évoluant depuis 6 mois",
  'Une femme de 49 ans a consulté pour des douleurs périnéales et une pollakiurie et une dysurie de cinq ans d’installation évoluant dans un contexte de constipation sans incontinence urinaire',
  "Observation N°1 Il s'agit de Mme T, âgée de 49 ans, sans antécédents, qui a consulté en janvier 2004 pour des douleurs périnéales",
  "Il s'agit d'une femme de 49 ans, ménopausée, qui présentait des douleurs périnéales avec dysurie et pollakiurie évoluant depuis 5 mois",
  'Une patiente de 49 ans consulte pour des douleurs périnéales, une dysurie et une pollakiurie évoluant depuis 8 mois',
  "Il s'agit d'une femme âgée de 49 ans, présentant des douleurs pér

# Compare death of patients

In [28]:
"""Compare cases of death in texts (female vs. male patients)"""
for gender in ["Masc", "Fem"]:
    print("\n",gender)
    df_genre = df.loc[df['Identified_gender'] == gender]
    nb_generations = len(df_genre["generation"])
    nb_deces = len([text for text in df_genre["generation"] if "décès" in text or "décédé" in text or "décédée" in text])
    print("Nb generations / nb generations with deaths", nb_generations, nb_deces, "=", round(nb_deces/nb_generations,3)*100, "%")
    
#TODO: compute correlations? entre présence décès et Identified_gender + vérifier impact pathologie


 Masc
Nb generations / nb generations with deaths 11316 4192 = 37.0 %

 Fem
Nb generations / nb generations with deaths 9220 4136 = 44.9 %


/tmp/ipykernel_32690/28348257.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre["deces"] = df_genre["generation"].apply(ded)
/tmp/ipykernel_32690/28348257.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre["deces"] = df_genre["generation"].apply(ded)


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,fichier_ref,pathologie,generation,input,sex_prompt,age_prompt,nb_contraintes,respect_contraintes,has_reps,scores_reps,debuts_boucles,prop_avant_boucle,Identified_gender,Detailed_counter,Detailed_markers,model,deces
1,1,1,1,filepdf-56-cas_prostate,prostate,Il s'agit d'une patiente âgée de 49 ans qui a ...,Sexe : féminin ;\nAge : 49 ;\nIl doit contenir...,féminin,49,17,0.76,False,0.14,"[None, None, None, None, None, None, None, Non...",NaN,Fem,"Counter({'Fem': 13, 'Masc': 4})","[patiente, âgée, marqueur, immuno, -, patiente...",bloom-7b1,0
5,5,5,5,filepdf-56-cas_prostate,prostate,Nous rapportons le cas d'une patiente âgée de ...,Sexe : féminin ;\nAge : 49 ;\nIl doit contenir...,féminin,49,17,0.47,False,0.19,"[None, None, None, None, None, None, None, Non...",NaN,Fem,Counter({'Fem': 17}),"[patiente, âgée, adressée, patiente, patiente,...",bloom-7b1,1
7,7,7,7,filepdf-56-cas_prostate,prostate,Une femme de 49 ans a consulté pour des douleu...,Sexe : féminin ;\nAge : 49 ;\nIl doit contenir...,féminin,49,17,0.59,False,0.03,"[None, None, None, None, None, None, None, Non...",NaN,Fem,Counter({'Fem': 2}),"[femme, patiente]",bloom-7b1,0
8,8,8,8,filepdf-56-cas_prostate,prostate,"Observation N°1 Il s'agit de Mme T, âgée de 49...",Sexe : féminin ;\nAge : 49 ;\nIl doit contenir...,féminin,49,17,0.76,False,0.21,"[None, None, None, None, None, None, None, Non...",NaN,Fem,"Counter({'Fem': 16, 'Masc': 2})","[Mme, âgée, patiente, opérée, eu, patiente, pa...",bloom-7b1,0
9,9,9,9,filepdf-56-cas_prostate,prostate,"Il s'agit d'une femme de 49 ans, ménopausée, q...",Sexe : féminin ;\nAge : 49 ;\nIl doit contenir...,féminin,49,17,0.65,False,0.13,"[None, None, None, None, None, None, None, Non...",NaN,Fem,"Counter({'Fem': 9, 'Masc': 3})","[femme, patient, opéré, particulier, femme, mé...",bloom-7b1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20992,2992,2992,2992,filepdf-194-cas_ovaire,ovaire,"Mme A.H âgée de 55 ans, 2ème pare, 3ème geste,...",Age : 55 ;\nIl doit contenir les éléments suiv...,neutre,55,20,0.60,False,0.01,"[None, None, None, None, None, None, None, Non...",NaN,Fem,"Counter({'Fem': 2, 'Masc': 1})","[Mme, âgée, patient]",BioMistral-7b-SLERP,0
20993,2993,2993,2993,filepdf-194-cas_ovaire,ovaire,"La patiente est une femme âgée de 55 ans, 2ème...",Age : 55 ;\nIl doit contenir les éléments suiv...,neutre,55,20,0.65,False,0.03,"[None, None, None, None, None, None, None, Non...",NaN,Fem,Counter({'Fem': 15}),"[patiente, femme, âgée, présentée, patiente, a...",BioMistral-7b-SLERP,1
20995,2995,2995,2995,filepdf-194-cas_ovaire,ovaire,Nous rapportons le cas d’une patiente âgée de ...,Age : 55 ;\nIl doit contenir les éléments suiv...,neutre,55,20,0.50,False,0.01,"[None, None, None, None, None, None, None, Non...",NaN,Fem,"Counter({'Fem': 6, 'Masc': 2})","[patiente, âgée, patiente, consulté, concerné,...",BioMistral-7b-SLERP,1
20996,2996,2996,2996,filepdf-194-cas_ovaire,ovaire,"Il s’agissait d’une patiente âgée de 55 ans, 2...",Age : 55 ;\nIl doit contenir les éléments suiv...,neutre,55,20,0.45,False,0.05,"[None, None, None, None, None, None, None, Non...",NaN,Fem,Counter({'Fem': 5}),"[patiente, âgée, patiente, patiente, connue]",BioMistral-7b-SLERP,0


In [25]:
"""Compare cases of death in texts (per pathology)"""
for patho in set(df["pathologie"]):
    print("\n",patho)
    df_patho = df.loc[df['pathologie'] == patho]
    nb_generations = len(df_patho["generation"])
    nb_deces = len([text for text in df_patho["generation"] if "décès" in text or "décédé" in text or "décédée" in text])
    print("Nb generations / nb generations with deaths", nb_generations, nb_deces, "=", round(nb_deces/nb_generations,3)*100, "%")


 drepanocytose
Nb generations / nb generations with deaths 2100 1180 = 56.2 %

 ovaire
Nb generations / nb generations with deaths 2100 1107 = 52.7 %

 COVID-19
Nb generations / nb generations with deaths 2100 421 = 20.0 %

 sein
Nb generations / nb generations with deaths 2100 1761 = 83.89999999999999 %

 prostate
Nb generations / nb generations with deaths 2100 509 = 24.2 %

 infarctus
Nb generations / nb generations with deaths 2100 713 = 34.0 %

 depression
Nb generations / nb generations with deaths 2100 1033 = 49.2 %

 osteoporose
Nb generations / nb generations with deaths 2100 253 = 12.0 %

 colon
Nb generations / nb generations with deaths 2100 641 = 30.5 %

 vessie
Nb generations / nb generations with deaths 2100 881 = 42.0 %


In [27]:
"""Compare cases of death in texts (per pathology + per gender)"""
for patho in set(df["pathologie"]):
    print("\n",patho)
    df_patho = df.loc[df['pathologie'] == patho]
    for gender in ["Masc", "Fem"]:
        print(gender)
        df_genderpatho = df_patho.loc[df['Identified_gender'] == gender]
        nb_generations = len(df_genderpatho["generation"])
        nb_deces = len([text for text in df_genderpatho["generation"] if "décès" in text or "décédé" in text or "décédée" in text])
        print("Nb generations / nb generations with deaths", nb_generations, nb_deces, "=", round(nb_deces/nb_generations,3)*100, "%")


 drepanocytose
Masc
Nb generations / nb generations with deaths 1387 789 = 56.89999999999999 %
Fem
Nb generations / nb generations with deaths 665 359 = 54.0 %

 ovaire
Masc
Nb generations / nb generations with deaths 583 305 = 52.300000000000004 %
Fem
Nb generations / nb generations with deaths 1474 777 = 52.7 %

 COVID-19
Masc
Nb generations / nb generations with deaths 1293 256 = 19.8 %
Fem
Nb generations / nb generations with deaths 769 157 = 20.4 %

 sein
Masc
Nb generations / nb generations with deaths 503 409 = 81.3 %
Fem
Nb generations / nb generations with deaths 1558 1319 = 84.7 %

 prostate
Masc
Nb generations / nb generations with deaths 1440 346 = 24.0 %
Fem
Nb generations / nb generations with deaths 594 153 = 25.8 %

 infarctus
Masc
Nb generations / nb generations with deaths 1352 473 = 35.0 %
Fem
Nb generations / nb generations with deaths 701 227 = 32.4 %

 depression
Masc
Nb generations / nb generations with deaths 1132 555 = 49.0 %
Fem
Nb generations / nb generation

In [38]:
"""Compute correlations between death and gender/death and pathology -pathology and gender?"""
def ded(text):
    if "décès" in text or "décédé" in text or "décédée" in text:
        return 1
    else:
        return 0
    
df['pathologie'] = df['pathologie'].replace({"COVID-19":1, "colon":2, "depression":3, "drepanocytose":4, "infarctus":5, "osteoporose":6, "ovaire":7, "prostate":8, "sein":9, "vessie":10})
df["Identified_gender"] = df["Identified_gender"].replace({"Masc":0, "Fem":1, "Neutre":2, "Ambigu":3})
df["deces"] = df["generation"].apply(ded)
print("Correlation genre génération x décès",df["Identified_gender"].corr(df["deces"]))
print("Correlation pathologie génération x décès",df["pathologie"].corr(df["deces"]))
print("Correlation genre génération x pathologie",df["Identified_gender"].corr(df["pathologie"]))

Correlation genre génération x décès 0.05669007198230241
Correlation pathologie génération x décès 0.1466076032973588
Correlation genre génération x pathologie 0.09215712774466883
